# Items demand utility

Import all the needed modules

In [1]:
import pandas as pd
from pandas import ExcelWriter
import numpy as np

Import from the statistics file the columns needed

In [2]:
excel_file = 'R12.xlsx'
df = pd.read_excel(excel_file, usecols = ['Item', 'Year month',
                                          'Sold qty', 'Used qty',
                                          'Not rep usage'])

Import from the master file the columns needed

In [3]:
master_file = 'Item master.xlsx'
df_master = pd.read_excel(master_file, usecols = ['Item no', 'Item name',
                                            'Product group descr'])

Import supply data for suppliers

In [4]:
excel_file = 'Supply.xlsx'
df_supply = pd.read_excel(excel_file, usecols = ['Item no', 'Supplier'])
df_supply.drop_duplicates(inplace=True)

Import price data for items

In [5]:
excel_file = 'MMS003.xlsx'
df_price = pd.read_excel(excel_file, usecols = ['Item no', 'Prod cost'])

Get the first word of the item column using str.split and str.get function

In [6]:
df['Item no'] = df['Item'].str.split().str.get(0)

Remove possible whitespaces in the figures

In [7]:
columns = ['Sold qty', 'Used qty', 'Not rep usage']
for column in columns:
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace(' ', '')
    df[column] = df[column].astype(float)

Get the demand

In [8]:
df['Demand'] = df['Sold qty'] + df['Used qty'] + np.where(df['Not rep usage'] > 0, df['Not rep usage'], 0)

Get the pivot table of the demand and year month and replace NaN with zeros

In [9]:
pivot_df = pd.pivot_table(df, values = 'Demand', index = 'Item no' , columns = 'Year month', aggfunc=np.sum)
pivot_df = pivot_df.fillna(0)
pivot_df = pd.DataFrame(pivot_df.to_records())

Total field for pivot_df

In [10]:
pivot_df['Total'] = pivot_df.sum(axis = 1, skipna = True)

Get the 'Item name' for each item. Merge it with the master_df

In [11]:
df_master = pd.merge(df_master, df_supply, how='left', on='Item no') # Suppliers
df_master = pd.merge(df_master, df_price, how='left', on='Item no') # Price
demand_df = pd.merge(df_master, pivot_df, how='left', left_on='Item no', right_on='Item no')
demand_df = demand_df.fillna(0)

Output the demand to Excel

In [12]:
name = 'Items Demand.xlsx'
writer = pd.ExcelWriter(name)
demand_df.to_excel(writer, index = False)
writer.save()